In [ ]:
#!pip install plotly
#!pip install chart-studio
#!pip install ipywidgets

In [1]:
import plotly.graph_objects as go
import plotly.express as px

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')
df.Date = pd.to_datetime(df.Date)
df = df.set_index('Date')[['AAPL.Open','AAPL.Close']]
df

,AAPL.Open,AAPL.Close
Date,,
2015-02-17,127.489998,127.830002
2015-02-18,127.629997,128.720001
2015-02-19,128.479996,128.449997
2015-02-20,128.619995,129.500000
2015-02-23,130.020004,133.000000
...,...,...
2017-02-10,132.460007,132.119995
2017-02-13,133.080002,133.289993
2017-02-14,133.470001,135.020004


In [6]:
def plot_corr(frequency, window):
    if not window or not frequency:
        return
    df_corr = (df
        .resample(frequency)
        .ffill()
        .rolling(window)
        .corr(pairwise=True)
        .unstack()['AAPL.Open']['AAPL.Close']
        )
    fig = px.line(x=df_corr.index, y=df_corr, title='Rolling correlation')
    fig.update_yaxes(range=(-1,1))
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    fig.show()

#plot_corr('D', 10)

In [7]:
from ipywidgets import interactive, widgets, interact

In [8]:
frequency_selector = widgets.ToggleButtons(
    options=['D', 'W', '30d'],
    value='D',
    description='Rolling window:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
    # icons=['check'] * 3
)

window_selector = widgets.IntSlider(
    value=10,
    min=1,
    max=30,
    step=1,
    description='Sample frequency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

_ = interact(plot_corr, frequency = frequency_selector,  window = window_selector)

interactive(children=(ToggleButtons(description='Rolling window:', options=('D', 'W', '30d'), tooltips=('Descr…